In [1]:
from configs import models, env, settings
from backend.llm.api import load_model, load_llm_langchain
from backend.prompts.templates import literature_qa_template

config_loaded = {"model_config": models, "env": env}
llm = load_llm_langchain(source='ollama', model_name='LLaMA-3', config=config_loaded)

[LLM Loader] Successfully initialized model 'llama3' from 'ollama'.


In [2]:
# PubMed search using Entrez (Biopython)
from Bio import Entrez
Entrez.email = env.get("email", "your@email.com") 

def search_pubmed(query, max_results=3):
    handle = Entrez.esearch(db="pubmed", term=query, retmax=max_results)
    record = Entrez.read(handle)
    id_list = record["IdList"]

    summaries = []
    for pubmed_id in id_list:
        fetch_handle = Entrez.efetch(db="pubmed", id=pubmed_id, retmode="xml")
        articles = Entrez.read(fetch_handle)
        article = articles['PubmedArticle'][0]
        title = article['MedlineCitation']['Article']['ArticleTitle']
        abstract = article['MedlineCitation']['Article']['Abstract']['AbstractText'][0]
        summaries.append(f"Title: {title}\nAbstract: {abstract}\n")
    return summaries


In [ ]:
question = "What are the latest treatment guidelines for hypertension in elderly patients?"
search_results = search_pubmed(question)

for i, doc in enumerate(search_results):
    print(f"🔹 Document {i+1}:\n{doc}\n")


🔹 Document 1:
Title: Long-term risk of cardiovascular mortality according to age group and blood pressure categories of the latest guideline.
Abstract: This study examined the association between the latest blood pressure (BP) classification and cardiovascular disease (CVD) mortality risk, using data from 70,570 individuals across 10 Japanese cohorts. Participants were stratified by age (40-64 and 65-89 years) and antihypertensive treatment use. BP was classified according to the 2019 Japanese Society of Hypertension Guidelines. During a follow-up period of approximately 10 years, 2304 CVD deaths occurred. Cox models demonstrated that CVD mortality risk increased stepwise with the BP category, with this association being especially pronounced in patients aged 40-64 years, where the Grade I hypertension group showed the highest population-attributable fraction (PAF). When the treated participants were included in the hypertension group, the overall PAF for CVD mortality was 41.1%. Simil

In [4]:
combined_context = "\n".join(search_results)
final_prompt = literature_qa_template.format(context=combined_context, question=question)

In [ ]:
response = llm.invoke(final_prompt)

print("📘 Answer:\n")
print(response.content)

📘 Answer:

Based on the provided research summaries, I can provide an answer to your question:

The latest treatment guidelines for hypertension in elderly patients are not explicitly stated in the provided abstracts. However, we can infer some information from the studies.

One study (Title: Long-term risk of cardiovascular mortality according to age group and blood pressure categories of the latest guideline) found that the Grade I hypertension group showed the highest population-attributable fraction (PAF) for CVD mortality in patients aged 40-64 years. This suggests that early-stage prevention and management of hypertension are important, especially in younger elderly patients.

Another study (Title: Cardiovascular events according to blood pressure thresholds recommended by ACC/AHA) compared different definitions of hypertension and found that the ACC/AHA definition identifies persons at elevated risk for future cardiovascular outcome. However, this study did not specifically focu